In [ ]:
from os import path

# Third-party
from astropy.io import ascii
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.stats import scoreatpercentile

from comoving_rv.log import logger
from comoving_rv.db import Session, Base, db_connect
from comoving_rv.db.model import (Run, Observation, TGASSource, SimbadInfo, PriorRV,
                                  SpectralLineInfo, SpectralLineMeasurement, RVMeasurement)

from gwb.data import TGASData
from gwb.fml import ln_H1_FML, ln_H2_FML

# import matplotlib as mpl
# mpl.rc('text', usetex=True)
# mpl.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]

In [ ]:
tgas = TGASData('../../../gaia-comoving-stars/data/stacked_tgas.fits')

In [ ]:
star = ascii.read('../../../gaia-comoving-stars/paper/t1-1-star.txt')
rave_stars = star[(star['group_size'] == 2) & (~star['rv'].mask)]
rave_stars = rave_stars.group_by('group_id')

Get only ones where both stars have RV measurements

In [ ]:
group_idx = np.array([i for i,g in enumerate(rave_stars.groups) if len(g) > 1])
rave_stars = rave_stars.groups[group_idx]

In [ ]:
class Worker(object):

    def __init__(self, Vinv, n_distance_samples, prior_weights):
        self.Vinv = np.array(Vinv)
        self.prior_weights = prior_weights
        self.n_distance_samples = n_distance_samples

    def work(self, star1, star2, v_scatter):     
        h1 = ln_H1_FML(star1, star2, Vinv=self.Vinv, v_scatter=v_scatter,
                       n_dist_samples=self.n_distance_samples, prior_weights=self.prior_weights)
        h2 = ln_H2_FML(star1, star2, Vinv=self.Vinv, v_scatter=v_scatter,
                       n_dist_samples=self.n_distance_samples, prior_weights=self.prior_weights)
        return h1, h2

In [ ]:
# MAGIC NUMBERs
n_distance_samples = 1024
Vinvs = np.array([np.eye(3)/15.**2,
                  np.eye(3)/30.**2,
                  np.eye(3)/50.**2])   # 3x3 inverse variances for gaussian prior on velocities
prior_weights = np.array([0.3, 0.55, 0.15])
v_scatter = 1. # km/s

worker = Worker(Vinv=Vinvs, n_distance_samples=n_distance_samples, prior_weights=prior_weights)

In [ ]:
names = ['group_id', 'lnH1_no_rv', 'lnH2_no_rv', 'lnH1_rv', 'lnH2_rv']
all_rows = []
for gid in np.unique(rave_stars['group_id']):
    rows = rave_stars[rave_stars['group_id'] == gid]
    
    if len(rows) != 2:
        print("skipping group {0} ({1})".format(gid, len(rows)))
        continue
        
    i1 = np.where(tgas._data['source_id'] == rows[0]['tgas_source_id'])[0][0]
    i2 = np.where(tgas._data['source_id'] == rows[1]['tgas_source_id'])[0][0]
    
    star1 = tgas[i1]
    star2 = tgas[i2]
    
    ln_H1_no_rv, ln_H2_no_rv = worker.work(star1, star2, v_scatter)

    star1._rv = rows[0]['rv']
    star2._rv = rows[1]['rv']
    star1._rv_err = rows[0]['erv']
    star2._rv_err = rows[1]['erv']
    ln_H1, ln_H2 = worker.work(star1, star2, v_scatter)
    
    all_rows.append((gid, ln_H1_no_rv, ln_H2_no_rv, ln_H1, ln_H2))
    
dtype = dict(names=names, formats=['i4']+['f8']*(len(names)-1))
tbl = np.array(all_rows, dtype)
tbl = Table(tbl)

In [ ]:
dv_pts = []
dv_15s = []
dv_meds = []
dv_85s = []
sep_2d = []
sep_3d = []

for gid in np.unique(rave_stars['group_id']):
    rows = rave_stars[rave_stars['group_id'] == gid]
    
    if len(rows) != 2:
        print("skipping group {0} ({1})".format(gid, len(rows)))
        continue
        
    i1 = np.where(tgas._data['source_id'] == rows[0]['tgas_source_id'])[0][0]
    i2 = np.where(tgas._data['source_id'] == rows[1]['tgas_source_id'])[0][0]
    
    star1 = tgas[i1]
    star2 = tgas[i2]
    
    icrs1 = star1.get_icrs(rows[0]['rv']*u.km/u.s)
    icrs2 = star2.get_icrs(rows[1]['rv']*u.km/u.s)
    
    # separations    
    sep_2d.append(icrs1.separation(icrs2))
    sep_3d.append(icrs1.separation_3d(icrs2))
    
    icrs1.set_representation_cls(coord.CartesianRepresentation, coord.CartesianDifferential)
    icrs2.set_representation_cls(coord.CartesianRepresentation, coord.CartesianDifferential)
    
    dv_pt = np.sqrt((icrs1.v_x-icrs2.v_x)**2 + 
                    (icrs1.v_y-icrs2.v_y)**2 + 
                    (icrs1.v_z-icrs2.v_z)**2)
    dv_pts.append(dv_pt)
    
    # Compute difference in 3D velocity for samples
    icrs1 = star1.get_icrs_samples(size=2**16, rv=rows[0]['rv']*u.km/u.s, 
                                   rv_err=rows[0]['erv']*u.km/u.s)
    icrs2 = star2.get_icrs_samples(size=2**16, rv=rows[1]['rv']*u.km/u.s, 
                                   rv_err=rows[1]['erv']*u.km/u.s)
    
    icrs1.set_representation_cls(coord.CartesianRepresentation, coord.CartesianDifferential)
    icrs2.set_representation_cls(coord.CartesianRepresentation, coord.CartesianDifferential)
    
    dv = np.sqrt((icrs1.v_x-icrs2.v_x)**2 + 
                 (icrs1.v_y-icrs2.v_y)**2 + 
                 (icrs1.v_z-icrs2.v_z)**2)
    
    dv_15, dv_med, dv_85 = scoreatpercentile(dv.value, [15, 50, 85])
    dv_15s.append(dv_15)
    dv_meds.append(dv_med)
    dv_85s.append(dv_85)

In [ ]:
tbl['dv'] = u.Quantity(dv_pts)
tbl['dv_15'] = dv_15s*u.km/u.s
tbl['dv_50'] = dv_meds*u.km/u.s
tbl['dv_85'] = dv_85s*u.km/u.s
tbl['sep_2d'] = u.Quantity(sep_2d)
tbl['sep_3d'] = u.Quantity(sep_3d)

In [ ]:
tbl['R_mu'] = tbl['lnH1_no_rv'] - tbl['lnH2_no_rv']
tbl['R_RV'] = tbl['lnH1_rv'] - tbl['lnH2_rv']

In [ ]:
tbl.write('RAVE_group_llr_dv_tbl.ecsv', format='ascii.ecsv')